# Assess ELA response to cumulative PDDs from observations and modeled conditions

In [ ]:
import os
import glob
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import xarray as xr
from tqdm.auto import tqdm
from scipy.stats import median_abs_deviation as MAD
from scipy.interpolate import RegularGridInterpolator
import sys
import seaborn as sns
# Suppress future warning from pandas
import warnings
warnings.filterwarnings("ignore")
import matplotlib
from sklearn.linear_model import LinearRegression

In [ ]:
base_path = '/Users/raineyaberle/Research/PhD/snow_cover_mapping/snow-cover-mapping-application/'
sys.path.append(os.path.join(base_path, 'functions'))
import model_analyze_utils as f

scm_path = '/Users/raineyaberle/Research/PhD/snow_cover_mapping/'

## Load glacier boundaries, ERA5-Land data, and compiled snow cover stats

In [ ]:
# -----Load glacier boundaries with climate clusters
aois_fn = os.path.join(scm_path, 'compiled_data', 'all_aois_climate_cluster.shp')
aois = gpd.read_file(aois_fn)
aois[['O1Region', 'O2Region']] = aois[['O1Region', 'O2Region']].astype(int)
print('All AOIs with climate clusters loaded from file.')

# -----Load ERA data
eras_fn = os.path.join(scm_path, 'compiled_data', 'all_era_data.csv')
eras = pd.read_csv(eras_fn)
# format dates as datetimes
eras['Date'] = pd.to_datetime(eras['Date'])
print('All ERA data loaded from file.')

# -----Load compiled snowlines
snowlines_fn = os.path.join(scm_path, 'compiled_data', 'all_snowlines.csv')
snowlines = pd.read_csv(snowlines_fn)
snowlines['datetime'] = pd.to_datetime(snowlines['datetime'], format='mixed')
print('All snowlines loaded from file.')
# snowlines

## Estimate and save modeled monthly transient ELAs at each site

In [ ]:
mod_elas_fn = os.path.join(scm_path, 'Rounce_et_al_2023', 'modeled_elas.csv')
if os.path.exists(mod_elas_fn):
    mod_elas = pd.read_csv(mod_elas_fn, index_col=0)
    print('Modeled ELAs loaded from file.')
else:
    
    # load binned model data
    bin_fns = sorted(glob.glob(os.path.join(scm_path, 'Rounce_et_al_2023', 'binned', '*.nc')))
    
    # remove binned file names for sites without snow cover observations
    aoi_ids = [x[7:] for x in aois['RGIId'].drop_duplicates().values]
    bin_fns = [x for x in bin_fns if os.path.basename(x)[0:7] in aoi_ids]
    
    # iterate over binned file names
    i=0
    for bin_fn in tqdm(bin_fns):
        # open binned data
        bin = xr.open_dataset(bin_fn)
        rgi_id = bin.RGIId.data[0] # grab RGI ID

        # subset data to 2013 on
        Itime = np.ravel(np.argwhere(bin.time.dt.year.data >= 2013))
        bin = bin.isel(time=Itime)
        
        # iterate over months
        elas = np.zeros(len(bin.time.data)) # initialize transient ELAs
        for j in range(0,len(bin.time.data)):
            # subset binned data to datetime
            bin_time = bin.isel(time=j)
            dt = bin.time.data[j]
            ba = bin_time.bin_massbalclim_monthly.data[0]
            h = bin_time.bin_surface_h_initial.data[0]
            # estimate transient ELA
            if any(ba < 0) & any(ba > 0):
                Iela = np.abs(bin_time.bin_massbalclim_monthly.data[0]).argmin()
                elas[j] = h[Iela]
            elif all(ba > 0):
                elas[j] = np.min(h)
            elif all(ba < 0):
                elas[j] = np.max(h)
            else:
                elas[j] = np.nan
                
        # compile in dataframe
        df = pd.DataFrame({rgi_id: elas}, index=bin.time.data)
        if i==0:
            mod_elas = df
        else:
            mod_elas[rgi_id] = elas
            
        i+=1

    # save to file
    mod_elas.to_csv(mod_elas_fn, index=True)
    print('Modeled transient ELAs saved to file:', mod_elas_fn)

mod_elas

## Fit linear trendlines to modeled and observed ELAs and PDDs

In [ ]:
# -----Modeled
# Add Year and month columns to ERA and modeled ELA data
eras['Year'] = eras['Date'].dt.year
eras['Month'] = eras['Date'].dt.month
mod_elas['Year'] = pd.DatetimeIndex(mod_elas.index).year
mod_elas['Month'] = pd.DatetimeIndex(mod_elas.index).month
# Resample ERA data to monthly resolution to match modeled data
eras_gb = eras.groupby(by=['site_name', 'Year', 'Month'])[['Cumulative_Positive_Degree_Days', 'Cumulative_Snowfall_mwe']].mean().reset_index()
# Remove dates outside 2013-2022
eras_gb = eras_gb.loc[(eras_gb['Year'] >= 2013) & (eras_gb['Year'] < 2023)]
# Remove dates after September
eras_gb = eras_gb.loc[eras_gb['Month'] < 9]
mod_elas = mod_elas.loc[mod_elas['Month'] < 9]
mod_elas.drop(columns=['Year', 'Month'], inplace=True)
years = eras_gb['Year'].values.astype(str)
months = ['0' + str(x) if x < 10 else str(x) for x in eras_gb['Month'].values]
eras_gb['Dates'] = [np.datetime64(year + '-' + month + '-01') for year, month in list(zip(years, months))]
dates = eras_gb['Dates'].drop_duplicates().values
# rearrange dataframe
i = 0
for rgi_id in tqdm(eras_gb['site_name'].drop_duplicates().values):
    if i==0:
        pdds_df = pd.DataFrame({rgi_id: eras_gb.loc[eras_gb['site_name']==rgi_id, 
                               'Cumulative_Positive_Degree_Days'].values}, index=dates)
        sf_df = pd.DataFrame({rgi_id: eras_gb.loc[eras_gb['site_name']==rgi_id, 
                              'Cumulative_Snowfall_mwe'].values}, index=dates)
    else:
        pdds_df[rgi_id] = eras_gb.loc[eras_gb['site_name']==rgi_id, 'Cumulative_Positive_Degree_Days'].values
        sf_df[rgi_id] = eras_gb.loc[eras_gb['site_name']==rgi_id, 'Cumulative_Snowfall_mwe'].values
    i+=1

In [ ]:
# Fit linear trendlines to PDDs + Snowfall = ELAs
fits_mod_df = pd.DataFrame()
for rgi_id in tqdm(mod_elas.columns):
    # subset data
    mod_elas_site = mod_elas[rgi_id].values
    pdds_site = pdds_df[rgi_id].values
    sf_site = sf_df[rgi_id].values
    # fit linear trendline
    X = np.concatenate([pdds_site.reshape(-1,1), sf_site.reshape(-1,1)], axis=1)
    y = mod_elas_site.reshape(-1,1)
    model = LinearRegression().fit(X, y)
    # save in dataframe
    fit_df = pd.DataFrame({'RGIId': [rgi_id], 
                           'O1Region': [aois.loc[aois['RGIId']==rgi_id, 'O1Region'].values[0]],
                           'O2Region': [aois.loc[aois['RGIId']==rgi_id, 'O2Region'].values[0]],
                           'Subregion': [aois.loc[aois['RGIId']==rgi_id, 'Subregion'].values[0]],
                           'cluster': [aois.loc[aois['RGIId']==rgi_id, 'cluster'].values[0]],
                           'clustName': [aois.loc[aois['RGIId']==rgi_id, 'clustName'].values[0]],
                           'coef_PDD': [model.coef_[0][0]],
                           'coef_Snowfall': [model.coef_[0][1]],
                           'score': [model.score(X, y)]})
    fits_mod_df = pd.concat([fits_mod_df, fit_df])
    # plot
    # fig, ax = plt.subplots()
    # ax.plot(X[:,0], y, '.')
    # ax.set_xlabel('$\Sigma$PDDs')
    # ax.set_ylabel('Transient ELA [m]')
    # plt.show()
fits_mod_df.reset_index(drop=True, inplace=True)
fits_mod_df

# Save
fits_mod_fn = os.path.join(scm_path, 'results', 'linear_fit_ela_modeled_pdd_snowfall.csv')
fits_mod_df.to_csv(fits_mod_fn, index=False)
print('Linear fits saved to file:', fits_mod_fn)
fits_mod_df

In [ ]:
# -----Observed
# Merge snowlines and ERA data
# Add Month column to snowlines
snowlines['Month'] = pd.DatetimeIndex(snowlines['datetime']).month.values
eras['Month'] = pd.DatetimeIndex(eras['Date']).month.values
# Remove observations after August
snowlines = snowlines.loc[snowlines['Month'] < 8]
eras = eras.loc[eras['Month'] < 8]
# Unify date columns for merging
snowlines['Date'] = snowlines['datetime'].values.astype('datetime64[D]')
eras['Date'] = eras['Date'].values.astype('datetime64[D]')
# Merge on site name and dates
merged = pd.merge(snowlines, eras, on=['site_name', 'Date'])
merged

In [ ]:
# Iterate over RGI IDs
fits_obs_df = pd.DataFrame()
for rgi_id in tqdm(merged['site_name'].drop_duplicates().values):
    merged_site = merged.loc[merged['site_name']==rgi_id]
    # Fit linear trendline
    X = merged_site[['Cumulative_Positive_Degree_Days', 'Cumulative_Snowfall_mwe']].values
    y = merged_site['ELA_from_AAR_m'].values
    model = LinearRegression().fit(X, y)
    # save in dataframe
    fit_df = pd.DataFrame({'RGIId': [rgi_id], 
                           'O1Region': [aois.loc[aois['RGIId']==rgi_id, 'O1Region'].values[0]],
                           'O2Region': [aois.loc[aois['RGIId']==rgi_id, 'O2Region'].values[0]],
                           'Subregion': [aois.loc[aois['RGIId']==rgi_id, 'Subregion'].values[0]],
                           'cluster': [aois.loc[aois['RGIId']==rgi_id, 'cluster'].values[0]],
                           'clustName': [aois.loc[aois['RGIId']==rgi_id, 'clustName'].values[0]],
                           'coef_PDD': [model.coef_[0]],
                           'coef_Snowfall': [model.coef_[1]],
                           'score': [model.score(X, y)]})
    fits_obs_df = pd.concat([fits_obs_df, fit_df])

fits_obs_df.reset_index(drop=True, inplace=True)

# Save
fits_obs_fn = os.path.join(scm_path, 'results', 'linear_fit_ela_observed_pdd_snowfall.csv')
fits_obs_df.to_csv(fits_obs_fn, index=False)
print('Linear fits saved to file:', fits_obs_fn)
fits_obs_df

## Plot scatterplot for comparison

In [ ]:
# Merge dataframes according to RGI ID
fits_merged_df = fits_mod_df.merge(fits_obs_df, 
                                   on=['RGIId', 'O1Region', 'O2Region', 'Subregion', 'cluster', 'clustName'], 
                                   suffixes=['_mod', '_obs'])
# Remove wacky values
for col in ['coef_PDD_mod', 'coef_PDD_obs', 'coef_Snowfall_mod', 'coef_Snowfall_obs']:
    fits_merged_df.loc[np.abs(fits_merged_df[col]) > 1e33, col] = np.nan

# Plot
fig, ax = plt.subplots(2, 1, figsize=(10,12))
for i, col in enumerate(['PDD', 'Snowfall']):
    sns.scatterplot(data=fits_merged_df, x=f'coef_{col}_mod', y=f'coef_{col}_obs', hue='Subregion', ax=ax[i])
    min_coef = np.nanmin(np.ravel(fits_merged_df[[f'coef_{col}_mod', f'coef_{col}_obs']].values))
    max_coef = np.nanmax(np.ravel(fits_merged_df[[f'coef_{col}_mod', f'coef_{col}_obs']].values))
    ax[i].plot(np.arange(min_coef, max_coef), np.arange(min_coef, max_coef), '-k')
    if i==0:
        units = col
    else:
        units = 'snow m.w.e.'
    ax[i].set_xlabel(f'Model coefficient [{units}/m]')
    ax[i].set_ylabel(f'Observed coefficient [{units}/m]')
    ax[i].grid()

# Add some annotations
ax[0].text(15, 5, 'Higher OBSERVED \nELA sensitivity \nto PDDs', 
           horizontalalignment='center', fontsize=12, fontweight='bold')
ax[0].text(10, 22, 'Higher MODELED \nELA sensitivity \nto PDDs', 
           horizontalalignment='center', fontsize=12, fontweight='bold')

plt.show()

# Save figure
fig_fn = os.path.join(scm_path, 'snow-cover-mapping-application', 'figures', 
                      'ela_sensitivity_modeled_observed_comparison.png')
fig.savefig(fig_fn, dpi=300)
print('Figure saved to file:', fig_fn)